# IPython para administración de sistemas Unix

Pueden listarse las funciones dentro de un módulo con la consola de `IPython` usando el **tabulador** cuando se importa el módulo, se escribe por consola y se añade un punto:

```python
import subprocess
subprocess.
```

Para conocer un ejemplo de uso de una de las funciones puede añadirse un `?` detrás de la función como en el ejemplo. Puede obtenerse más ayuda si se introducen dos `??`.

In [ ]:
subprocess.getoutput??

Es buena idea cuando se crean scripts de shell con Python envolver todas las funciones que representan un módulo en un `main` de tal manera que se ejecuten secuencialmente todas las funciones. De esta manera se tienen por un lado llamadas individuales a las funciones y por otro el conjunto de todas ellas. A esta función `main` se le denomina **flujo de control**.

In [ ]:
import subprocess

# Comando 1
def uname_func():
    
    uname = "uname"
    uname_arg = "-a"
    print("Gathering system informatiokn with %s command:\n" % uname)
    subprocess.call([uname, uname_arg])
    
    
#Comando 2
def disk_func():
    
    diskspace = "df"
    diskspace_arg = "-h"
    print("Gathering diskpace information %s command:\n" % diskspace)
    subprocess.call([diskspace, diskspace_arg])

    
# Función principal que invoca otras funciones.
def main():
    uname_func()
    disk_func()
    
main()

## Funciones mágicas

Para listar las funciones mágicas introducir: `lsmagic`. También puede utilizarse el tabulador si se pone por delante el caracter especial `%`. Puede desplegarse una ayuda si se introduce el comando: `magic`.

In [ ]:
lsmagic

Un ejemplo de una función mágica, permite crear `alias` como en el ejemplo:

```python
%alias d ls -F
```

Para desactivarlo, utilizar la función mágica `%unalias`.

In [ ]:
%alias d ls -F

Por ejemplo, pueden listarse las funciones y módulos interactivos que se están usando en la sesión de IPython con la **función mágica** `%who`o `%whos` con algo extra de información. Por ejemplo, hasta esta celdilla se han ejecutado otras celdas que contienen información. La lista se muestra en la siguiente celdilla.

In [ ]:
%whos

Una guía rápida de las funciones mágicas puede sacarse con `%quickref`. Un ejemplo se muestra a continuación.

In [ ]:
%quickref

Un ejemplo de alias útil para gestionar un sistema Unix pueden ser, como se dijo antes, los alias. Si, por ejemplo, se quiere utilizar el comando `netstat -plunt` de manera más rápida, puede hacerse u alias como el siguiente:

In [ ]:
%alias nss netstat -plunt

In [ ]:
nss

Además, puede utilizarse las metodologías **hacer-nada** o **hacer-todo** donde se le pueden pasar o no argumentos a un alias o comando e interpretarse de distinta manera. Teniendo por ejemplo el alias:

In [ ]:
%alias achoo echo first: "|%s|", second "|%s|"

In [ ]:
achoo foo bar

Para hacerlo con o sin argumentos:

In [ ]:
%alias achoo echo "|%l|"

In [ ]:
achoo

Otra forma de ejecutar un comando de shell es colocando el símbolo `!` delante. La salida de un comando de shell puede almacenarse en una lista, como por ejemplo.

In [ ]:
user = 'nachoaz'
process = 'bash'
l = !ps aux | grep $user | grep $process
l # Imprimir la variable 'l'.

## cd y marcadores 

## Bookmarks

Crear marcadores se puede hacer de diferentes maneras. Una de ellas es la siguiente: con la directiva `bookmark t` estando, por ejemplo, en el directorio `/tmp` se crea un marcador en ese directorio.

In [ ]:
cd /tmp

In [ ]:
bookmark t

Otra manera de crear un marcador requiere escribir un poco más.

In [ ]:
bookmark h /home/

Pueden listarse los marcadores con la siguiente opción. **Nota:** Los marcadores se conservan entre sesiones de IPython por lo que no es necesario cargarlos de nuevo.

In [ ]:
%bookmark -l

Para eliminar un marcador se utiliza el argumento `-d` como sigue:

In [ ]:
bookmark -d h

In [ ]:
bookmark -l

## dhist

El comando `dhist` no solamente guarda el histórico de la sesión actual sino de sesiones anteriores.

In [ ]:
dhist

Puede pasarsele argumentos para especificar, por ejemplo, la salida máxima del comando, haciendo una lista de los últimos 5 elementos de la historia con: `dhist 5` o un rango entre valores con `dhist 3 7` 

## Aplicación de Variable

Hasta ahora había una clara diferencia entre los comandos de shell e IPython pero vamos a ver que se pueden cruzar. Se va a tomar un valor que obtendremos de Python y pasarlo a la Shell. Aunque el ejemplo es "poco realista" dado que es raro crear 10 archivos con la fecha dentro, nos da una idea de la utilidad en el cruce de información entre ambos lenguajes.

In [ ]:
# {} = elemento actual de la lista
for i in range(3):
    !date > {i}.txt

In [ ]:
!ls -l

!cat 0.txt

Eliminamos los archivos creados en el apartado anterior.

In [ ]:
!rm *.txt

## Procesado de cadena

mOtra herramienta muy interesante que ofrece IPython es procesar en cadena el resultado de los comandos de shell del sistema. Si, por ejemplo, se quiere ver el PID de todos los procesos que pertenecen a un usuario se podría hacer de la siguiente manera: 

`ps aux | awk '{if ($1 == "myuser") print $2}'`

Si ahora intentamos repetir el proceso usando IPython hay que hacerlo en una pequeña secuencia que es igualmente legible.

In [ ]:
ps = !ps aux
ps.grep('mysuer', prune=False).fields(0, 1, 8)

El argumento `prune` se utiliza para que el retorno del argumento sea lo contrario a lo que se quiere o lo que se quiere. Por ejemplo, si quisieramos que el retorno fuera 'todos los usuarios que no sean el nuestro' (`root` por ejemplo) se pondría: `ps.grep('myuser', prune=True)`, es decir, __excluir__ del filtrado. De esa manera devuelve todos los usuarios que no sean `myuser`.

Con el método `fields()` se le especifican a `grep` las columnas que queremos devolver una vez se ejecute el comando. Hace las veces cuando se imprime una columna con `awk` utilizando `$1`, `$2`. En el ejemplo anterior, las columnas `0`, `1` y `8` hacen referencia a `USERNAME`, `PID` y `STARTTIME`.

Otro elemento muy interesante es, seguido del método `fields()` que va seguido del método `grep()` y que ambos devuelven objetos del mismo tipo, se puede añadir el elemento `s` que accede directamente a la lista en el proceso. Esto proporciona una cadena de PID separados por espacio con la que la Shell ya puede trabajar y de podría, por ejemplo hacer `kill $pids` desde dentro de IPython.

Por último, el método `grep()` adopta un último parámetro opcional denominado `field`. Si se especifica, los criterios de búsqueda tienen que coincidir con ese campo para que se incluya en el resultado. Esto coincidía con las dilas exactas que se quieren pero mostraba toda la fila. Para llegar solo al PID hay que añadirle `fields(1)` detrás.

In [ ]:
ps = !ps aux
ps.grep('nachoaz', field=0).fields(1)

## Perfil sh

Un perfil es simplemente un conjunto de datos de configuracioń que se carga cuando se inicia IPython. Puede personalizarse un número de perfiles para hacer qye IPython se lleve a cabo de diferentes formas dependiendo de las necesidades de la sesión. Para invocar un perfil específico se utiliza la opción de línea de comando `-p` y se especifica el perfil que se quiere utilizar.

El perfil `sh` es el más utilizado dado que incorpora ciertos elementos de configuración de modo que IPython se convierte en una shell del sistema más amigable.

Dos ejemplos de valores de configuración que son diferentes del perfil estándar de IPython son que `sh` muestra el directorio actual y `rehash` y su `PATH` de modo que tiene acceso instantáneo a todos los mismos ejecutables que tednría en `bash`.

`sh` permite además la extensión `envpersist`. Permite modificar varias variables de entorno fácilmente y de forma persistente para su perfil `sh` IPython sin necesidad de actualizar el `.bash_profile` o `.bashrc`.

En un terminal introducimos:

```bash
ipython --profile=sh

Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
Type 'copyright', 'credits' or 'license' for more information
IPython 7.5.0 -- An enhanced Interactive Python. Type '?' for help.

IPython profile: sh

In [1]:                                                                                                           
```

Y tenemos la consola de IPython con el perfil de `sh` cargado.

## Recopilación de información

### page

Si un objeto no entra en la pantalla, con la función mágica `page` puede obtenerse un paginador. El predeterminado por el sistema es `less`. Un ejemplo sería el siguiente que si se compara con la función `page` permite tenerlo de manera completa en lugar de tener que hacer _scroll_ para ver todo el contenido en la celdilla.

In [ ]:
p = !ps aux
print(p)

In [ ]:
%page -r p

### pinfo

Proporciona información como tipo, clase base, espacio de nombre y _docstring_. Si tenemos un módulo que contiene una serie de clases se puede solicitar información del propio módulo.

In [ ]:
import time
import cv2

In [ ]:
%pinfo time

Puede soliitarse información acerca de:
- Del propio módulo:
    ```python
    pinfo module
    ```
- De una clase del módulo:
    ```python
    pinfo module.foo
    ```
- De una instancia de una de las clases:
    ```python
    f = module.Foo()
    pinfo p
    ```
`pinfo` tiene la misma funcionalidad que `?` delante del nombre de un objeto como se vió al principio.

### psearch

Esta función mágica buscará objetos Python por nombre con la ayuda de comodines. Declarando una secuencia de objetos:

In [ ]:
a = 1
aa = "one"
b = 2
bb = "two"
c = 3
cc = "three"

In [ ]:
psearch a*

Los objetos que se han encontrado además de `a`, `aa`, `b`, etc son objetos incorporados. Una alternativa rápida a `psearch` es cambiarlo por `?`. Puede usarse `-s` para buscar, `-e` para excluir en un determinado espacio de nombre. Estos espacios de nombre incluyen `builtin`, `user_local`, `user_global`, `internal` y `alias`. Por defecto, `psearch` busca en los dos primeros. Para buscar explícitaente en `user_local` puede pasar una opción a `psearch` para excluir buscar el espacio de nombre `builtin`. La ruta de búsqueda predeterminada para `psearch` es: `builtin` y `user_local` por lo que si especificamos `-s user` buscar `builtin` y `user_local` seguirá siendo lo que hemos pedido que haga.

In [ ]:
%psearch -e builtin a*

In [ ]:
%psearch -s user_local a*

`psearch` permite también buscar tipos específicos de objetos. Si, por ejemplo, se busca el espacio de nombres `user_local` en busca de enteros:

In [ ]:
%psearch a*.* int

Y si buscamos cadenas:

In [ ]:
%psearch -e builtin * string

### who

IPython proporciona un número de facilidades para listar todos los objetos interactivos. 

In [ ]:
%who

O especificando el tipo:

In [ ]:
#%who int
%who str

### who_ls

Es similar a `who` pero retorna una lista con la que se pueden hacer otras operaciones:

In [ ]:
%who_ls str

In [ ]:
# con _ se accede a la última lista de nombres.
# Es: el último resultado
for i in _:
    print(i)

### whos

Es similar a `who` pero imprime información que no saca la anterior función mágica.

In [ ]:
%whos

### macro

`macro` permite definir un bloque de código que se ejecutará más adelante en línea con cualquier código que se esté trabajando. Es diferente de crear funciones o métodos. La macro se hace consciente del contexto actual del código. Si tiene un conjunto común de pasos de procesado que frecuentemente se ejecuta en todos los archivos, se puede crear una `macro` para trabajar en los archivos. Por ejemplo

In [ ]:
dirlist = []

In [ ]:
for f in dirlist:
    print("Working on", f)
    print("done with", f)
    print("moving %s to %s.done" % (f, f))
    print("*" * 40)

Sintaxis: `macro nombre_macro rango_de_lineas`.

Donde el rango de líneas es la lista de las lineas de la historia que se quiere incorporar al `macro`.

_Nota: Hay que fijarse en el número que está encerrado en la celdilla, ya que el código puede editarse y guardarse después. Una modificación en la celdilla anterior eleva otro número de ejecución y ese será el que queramos guardar. De igual modo pueden establecerse rangos de celdillas separado por `-`_.

In [ ]:
%macro procdir 33

In [ ]:
dirlist = ['a.txt', 'b.txt', 'c.txt']

In [ ]:
procdir

# store

Para almacenar los macros guardados se utiliza la función `store` seguido del nombre que se le quiera dar a la _macro_.

In [ ]:
%store procdir

### reset

Para eliminar todas las variables del espacio de nombres se utiliza la función `reset`. En el siguiente ejemplo puede verse como se crean unas vatiables que se eliminan posteriormente utilizando `reset`.

In [ ]:
a = 1
b = 2
c = 3
%whos

In [ ]:
%reset

In [ ]:
%whos

### run

Con la función `run` se puede ejecutar un archivo que se especifique a IPython. Esto permite trabajar en un módulo Python en un editor de texto externo y comprobar interactivamente los cambios que se hace en él desde IPython o Jupyter como se está haciendo en este documento. Después de ejecutar el programa, se devuelve a la shell de IPython. La sintaxis para ejecutar es: `run opciones_archivo arguments`. Existen diferentes opciones:
 
- `-n`: hace que la variable `__name__` del módulo se establezca no en `__main__` sino en su propio nombre. Esto hace que el módulo se ejecute como si simplemente se importara.
- `-i`: Ejecuta el módulo en el espacio de nombre actual de IPython y proporciona al módulo acceso a todas las variables definidas.
- `-e`: Hace que IPython ignore llamadas a `sys.exit()` y excepciones `SystemExit`. Si algo de esto ocurre, IPython simplemente continúa con el programa.
- `-t`: Hace que IPython imprima información sobre la longitud de tiempo que tardó el módulo en ejecutarse.
- `-d`: hace que el módulo especificado se ejecuta bajo el depurador Python (pdb).
- `-p` : Ejecuta el módulo especificado bajo el creador de perfil Python.

### save

Para guardar las líneas de entrada especificadas en el archivo de salida se utiliza la sintaxis: 

```python
%save opciones nombre_archivo lineas
```

La única opción de la función `save` es `-r` que especifica que se guardará el archivo de entrada sin formato (raw) en lugar de traducido.

### rep

Esta función toma un número de parámetros que se podría encontrar de utilidad. Utilizar `rep` sin parámetros toma el último resultado queq se procesó (como el `ans` de las calculadoras científicas) y sitúa una representación de ello en la siguiente entrada.

In [4]:
def format_str(s):
    return "str(%s)" % s

In [5]:
format_str(1)

'str(1)'

In [8]:
%rep

In [ ]:
str(1)

## Texto

Métodos `lstrin()`, `rstrip()`. `strip()`

- `lstrip()`: Utilizarlo sin argumentos elimina todo espacio en blanco que esté al **principio** de la cadena y devuelve una nueva cadena (no modifica la original).
- `rstrip()`: Sin argumentos elimina todo espacio en blanco al **final** de la cadena y devuelve una nueva cadena.
- `strip()`: Elimina espacio en blanco al **principio** y **final** de la cadena y devuelve una nueva cadena.

Ejemplos:

In [1]:
spacious_string = "\n\t Texto sin espacios \t\r"
print(spacious_string)


	 Texto sin espacios 	


In [3]:
print(spacious_string.lstrip()) # Elimina los espacios al principio de la cadena (preserva el \t\r)
print(spacious_string.rstrip()) # Elimina los espacios al final de la cadena (preserva el \n\t)
print(spacious_string.strip())  # Elimina los espacios al principio y al final de la cadena

Texto sin espacios 	

	 Texto sin espacios
Texto sin espacios


Estos métodos con _strings_ también son útiles para eliminar partes de una cadena, como por ejemplo parsear un `xml`.

In [16]:
##### xml_tag = "<some_tag>"
xml_tag.lstrip("<")

'some_tag>'

In [6]:
xml_tag.lstrip(">") # Como al principio de la cadena no existe ese caracter, el resutado es el mismo

'<some_tag>'

In [7]:
xml_tag.rstrip(">")

'<some_tag'

In [8]:
xml_tag.strip("<").strip(">")

'some_tag'

In [9]:
xml_tag.strip("<>")

'some_tag'

In [10]:
gt_it_str = "<><><>gt lt str<><><>"

In [12]:
gt_it_str.strip("<>")

'gt lt str'

También hay que tener cuidado con `strip()` dado que elimina **todas** las coincidencias que encuentre en la cadena y no patrones como puede parecer. En el siguiente ejemplo puede verse claramente.

In [13]:
foo_str = "<fooooooooo>hola<foo>"

In [15]:
foo_str.strip("<foo>")

'hola'

## Expresiones regulares (re)

El patrón que buscaremos tiene que coincidir con dos llaves abiertas consecutivas seguidas por cualquier texto ( o no texto) seguido por dos llaves cerradas consecutivas.

In [3]:
import re

re_string = "{{(.*?)}}"
some_string = "Esta cadena contiene {{palabras}} dentro de {{corchetes}} para mostrar un {{ejemplo}} de {{expresión regular}}"

# findall retorna una lista con las coincidencias en la expresión regular
for match in re.findall(re_string, some_string):
    print("COINCIDENCIAS-> ", match)

COINCIDENCIAS->  palabras
COINCIDENCIAS->  corchetes
COINCIDENCIAS->  ejemplo
COINCIDENCIAS->  expresión regular


In [2]:
re.findall(re_string, some_string)

['palabras', 'corchetes', 'ejemplo', 'expresión regular']

Otra manera de trabajar con expresiones regulares en Python es mediante la creación de un objeto de **expresión regular compilada** y utilizar los métodos de ese objeto.

Una **expresión regular compilada** es un objeto que se creó al pasar un patrón a `re.compile()`. Contiene un número de métodos de expresión regular que se crearon al pasar un patrón a `re.compile()`. Existen dos diferencias principales entre utilizar los ejemplos compilados y no compilados. Primero, en lugar de mantener una referencia al patrón de expresión regular "`{{(.*?)}}`" se crea un objeto de expresión regular compilada y se utiliza el patrón para crearlo. En segundo lugar, en lugar de invocar a `findall()` del módulo `re` se invoca `findall()` en el objeto de expresión regular compilada.

In [4]:
import re

re_obj = re.compile("{{(.*?)}}")
some_string = "Esta cadena contiene {{palabras}} dentro de {{corchetes}} para mostrar un {{ejemplo}} de {{expresión regular}}"

for match in re_obj.findall(some_string):
    print("COINCIDENCIAS-> ", match)

COINCIDENCIAS->  palabras
COINCIDENCIAS->  corchetes
COINCIDENCIAS->  ejemplo
COINCIDENCIAS->  expresión regular


El segundo método (expresión regular compilada) es mucho más eficiente en términos de tiempo de búsqueda por lo que es recomendable su uso en vez de la primera opción.